### Problem 1: Nurse Scheduling with Preferences and Limits

You must assign nurses to shifts over a week.

- **Data**:
    - 7 days, 3 shifts/day (morning, afternoon, night)
    - 6 nurses
    - Each nurse can work at most 5 shifts a week and no more than one per day
    - No night shift followed by a morning shift the next day
    - Each shift must be covered by exactly one nurse
    - Nurses have preferences (0–10 score) for each shift
- **Objective**: Maximize total preference score across the week.

---

In [1]:
import gurobipy as gp
from gurobipy import GRB

def solve_nurse_scheduling(preferences):
    """
    Solves the nurse scheduling problem using Gurobipy.

    Args:
        preferences (dict): A dictionary where keys are (nurse, day, shift) tuples
                            and values are the preference scores (0-10).
                            Example: preferences[('N1', 'Mon', 'Morning')] = 8

    Returns:
        dict: A dictionary of assigned shifts if a solution is found,
              otherwise None.
    """

    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    shifts = ['Morning', 'Afternoon', 'Night']
    nurses = [f'N{i}' for i in range(1, 7)] # N1, N2, ..., N6 as Nurse Numbers

    max_shifts_per_week = 5
    max_shifts_per_day = 1

    model = gp.Model("Nurse_Scheduling")

    # --- Decision Variables ---
    # x[n, d, s] is 1 if nurse n works on day d, shift s; 0 otherwise
    x = model.addVars(nurses, days, shifts, vtype=GRB.BINARY, name="x")

    # --- Objective Function ---
    # Maximize total preference score
    model.setObjective(
        gp.quicksum(preferences.get((n, d, s), 0) * x[n, d, s]
                    for n in nurses for d in days for s in shifts),
        GRB.MAXIMIZE
    )

    # --- Constraints ---
    # 1. Each shift must be covered by exactly one nurse
    for d in days:
        for s in shifts:
            model.addConstr(
                gp.quicksum(x[n, d, s] for n in nurses) == 1,
                name=f"Shift_Coverage_{d}_{s}"
            )

    # 2. Each nurse can work at most max_shifts_per_week (5) shifts a week
    for n in nurses:
        model.addConstr(
            gp.quicksum(x[n, d, s] for d in days for s in shifts) <= max_shifts_per_week,
            name=f"Max_Shifts_Per_Week_{n}"
        )

    # 3. Each nurse can work at most max_shifts_per_day (1) shift per day
    for n in nurses:
        for d in days:
            model.addConstr(
                gp.quicksum(x[n, d, s] for s in shifts) <= max_shifts_per_day,
                name=f"Max_Shifts_Per_Day_{n}_{d}"
            )

    # 4. No night shift followed by a morning shift the next day
    for n in nurses:
        for i in range(len(days) - 1): # Iterate up to the second to last day
            today = days[i]
            next_day = days[i+1]
            model.addConstr(
                x[n, today, 'Night'] + x[n, next_day, 'Morning'] <= 1,
                name=f"No_Night_Morning_Overlap_{n}_{today}"
            )

    model.optimize()

    if model.status == GRB.OPTIMAL:
        print("\nOptimal solution found:")
        assigned_shifts = {}
        for n in nurses:
            for d in days:
                for s in shifts:
                    if x[n, d, s].x > 0.5: # Check if the variable is 1 (assigned)
                        assigned_shifts[(n, d, s)] = True
                        print(f"  Nurse {n} works on {d} - {s} shift")
        print(f"\nTotal Preference Score: {model.objVal}")
        return assigned_shifts
    elif model.status == GRB.INFEASIBLE:
        print("Model is infeasible.")
        return None
    elif model.status == GRB.UNBOUNDED:
        print("Model is unbounded.")
        return None
    else:
        print(f"Optimization ended with status {model.status}")
        return None

# --- Example Usage ---
if __name__ == "__main__":
    # Example Preference Data (replace with your actual data)
    # This is a random example to demonstrate the code.
    # In a real scenario, this would come from a database or input file.
    import random

    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    shifts = ['Morning', 'Afternoon', 'Night']
    nurses = [f'N{i}' for i in range(1, 7)]

    sample_preferences = {}
    for n in nurses:
        for d in days:
            for s in shifts:
                sample_preferences[(n, d, s)] = random.randint(0, 10) # Random preference for demonstration

    # You can also set specific preferences like this:
    # sample_preferences[('N1', 'Mon', 'Morning')] = 10
    # sample_preferences[('N2', 'Mon', 'Morning')] = 5

    scheduled_shifts = solve_nurse_scheduling(sample_preferences)

    if scheduled_shifts:
        # You can further process or display the scheduled_shifts dictionary
        pass

Set parameter Username
Set parameter LicenseID to value 2685582
Academic license - for non-commercial use only - expires 2026-07-08
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 105 rows, 126 columns and 450 nonzeros
Model fingerprint: 0xf532caf0
Variable types: 0 continuous, 126 integer (126 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 116.0000000
Presolve time: 0.00s
Presolved: 105 rows, 126 columns, 450 nonzeros
Variable types: 0 continuous, 126 integer (126 binary)

Root relaxation: objective 1.880000e+02, 58 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

---

### Problem 2: Production with Setup Times and Costs

A factory produces 4 products over 5 time periods.

- **Data**:
    - Each product requires different machine time and raw materials
    - Each switch to a new product has a fixed setup cost
    - Daily capacity for time and raw material
    - Storage cost for leftover products
    - Product demands per period
- **Objective**: Minimize total production + setup + storage costs.

---